In [1]:
%load_ext autoreload
%autoreload 2

import keras
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from Jlayers import SimpleLayer
from utilsSimpleConv2D import*
from SpectralLayer import Spectral
from spectralconvolutions import *
from tensorflow.keras.layers import Dense,Layer
from typing import Tuple,List,Any,Dict
from tensorflow.python.keras import activations, initializers, regularizers, constraints

In [2]:
## Data

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train, x_test=x_train.reshape(-1,28,28,1), x_test.reshape(-1,28,28,1)

x_train, x_test = x_train / 255.0, x_test / 255.0

flat_train = np.reshape(x_train, [x_train.shape[0], 28 * 28])
flat_test = np.reshape(x_test, [x_test.shape[0], 28 * 28])

In [4]:
## Parametres

In [7]:
spectral_config={
    'is_base_trainable' : False,
    'is_diag_start_trainable' : False,
    'is_diag_end_trainable' : True,
}

In [8]:
## Train phi

In [ ]:
accuracy=list()
parameters_one={ "use_lambda_out":False,
                 "use_lambda_in":False,
                 "trainable_SM_kernel":True,
                 "use_bias":False,
                 "kernel_initializer":"glorot_uniform"
           }

N2=[20,80,100,500,800,1000,1500,2000]
epochs = 10
batch_size=512

for i in range(len(N2)):
    print(f"Iteration {i}: nombre de neurones: {N2[i]}")

    model1 = tf.keras.Sequential()
    model1.add(tf.keras.layers.Input(shape=(28,28,1)))
    
    
    model1.add(SpectralConv2D_one(filters=1, kernel_size=3, strides=1, padding="VALID", activation="relu",**parameters_one))
    model1.add(keras.layers.MaxPooling2D((2,2)))
    model1.add(keras.layers.Dropout(0.2))       
                       
    model1.add(keras.layers.Flatten())     
    
    model1.add(Spectral(N2[i], **spectral_config,use_bias=True, activation='relu'))
    model1.add(keras.layers.Dropout(0.5))
    model1.add(Spectral(10, **spectral_config,use_bias=False, activation='softmax'))
               
    opt = tf.keras.optimizers.Adam(learning_rate=0.03)
    model1.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=0)
    accuracy.append(model1.evaluate(x_test,y_test,batch_size=32,verbose="auto"))

Iteration 0: nombre de neurones: 20
313/313 [==============================] - 5s 15ms/step - loss: 1.9340 - accuracy: 0.3838
Iteration 1: nombre de neurones: 80
313/313 [==============================] - 5s 16ms/step - loss: 1.5743 - accuracy: 0.5207
Iteration 2: nombre de neurones: 100
313/313 [==============================] - 5s 15ms/step - loss: 1.5481 - accuracy: 0.5096
Iteration 3: nombre de neurones: 500
313/313 [==============================] - 5s 15ms/step - loss: 0.7116 - accuracy: 0.8278
Iteration 4: nombre de neurones: 800
313/313 [==============================] - 5s 15ms/step - loss: 0.5827 - accuracy: 0.8615
Iteration 5: nombre de neurones: 1000


In [ ]:
score1=list()
for i in range(len(N2)):
    score1.append(accuracy[i][1])